In [1]:
import sys

sys.path.append("..")

from data_loading import *


In [2]:
n = 10000000
estimate_quantile = 0.9935

means = {"U": 5.589, "V": 0.018}
stds = {"U": 9.832, "V": 3.232}


In [4]:
data = pd.read_feather(f"subset/UV-{estimate_quantile}-{n}.ft")
df = data.copy()

NORMALIZED = False
GEOGRAPHICAL = False
CYCLIC_TIME = False

data.head()


,time,lev,lat,lon,U_est,V_est,U,V
0,2452,0,41,223,26.390625,3.857422,26.421875,3.845703
1,766,0,174,458,-2.804688,-2.136719,-2.679688,-1.933594
2,869,0,297,468,10.500000,-2.021484,10.625000,-1.998047
3,2272,0,26,264,7.726562,3.412109,7.730469,3.423828
4,2787,0,266,269,13.054688,-2.029297,13.171875,-2.205078


In [5]:
NORMALIZED = True

df["lat"] = (data["lat"] * 0.5) - 90
df["lat"] /= 90
df["lon"] = (data["lon"] * 0.625) - 180
df["lon"] /= 180
df["lev"] /= 36

df["lev"] = df["lev"].astype("float16")
df["lat"] = df["lat"].astype("float16")
df["lon"] = df["lon"].astype("float16")

df["U"] -= means["U"]
df["U"] /= stds["U"]

df["V"] -= means["V"]
df["V"] /= stds["V"]

df["U_est"] -= means["U"]
df["U_est"] /= stds["U"]

df["V_est"] -= means["V"]
df["V_est"] /= stds["V"]

df["day %"] = ((data["time"] % 8) / 8).astype("float16")
df["year %"] = (data["time"] / 2920).astype("float16")
del df["time"]

df.head()


,lev,lat,lon,U_est,V_est,U,V,day %,year %
0,0.0,-0.772461,-0.225708,2.115234,1.187500,2.119141,1.184570,0.500,0.839726
1,0.0,-0.033325,0.590332,-0.853516,-0.666504,-0.840820,-0.603516,0.750,0.262329
2,0.0,0.649902,0.625000,0.499512,-0.630859,0.512207,-0.623535,0.625,0.297603
3,0.0,-0.855469,-0.083313,0.217407,1.049805,0.217773,1.053711,0.000,0.778082
4,0.0,0.477783,-0.065979,0.759766,-0.633301,0.771484,-0.687500,0.375,0.954452


In [6]:
CYCLIC_TIME = True

df["sin_day"] = np.sin(df["day %"] * np.pi * 2).astype("float16")
df["cos_day"] = np.cos(df["day %"] * np.pi * 2).astype("float16")
df["sin_year"] = np.sin(df["year %"] * np.pi * 2).astype("float16")
df["cos_year"] = np.cos(df["year %"] * np.pi * 2).astype("float16")

df.head()


,lev,lat,lon,U_est,V_est,U,V,day %,year %,sin_day,cos_day,sin_year,cos_year
0,0.0,-0.772461,-0.225708,2.115234,1.187500,2.119141,1.184570,0.500,0.839726,0.000000,-1.000000,-0.845215,0.534180
1,0.0,-0.033325,0.590332,-0.853516,-0.666504,-0.840820,-0.603516,0.750,0.262329,-1.000000,-0.000000,0.997070,-0.077393
2,0.0,0.649902,0.625000,0.499512,-0.630859,0.512207,-0.623535,0.625,0.297603,-0.707031,-0.707031,0.955566,-0.294678
3,0.0,-0.855469,-0.083313,0.217407,1.049805,0.217773,1.053711,0.000,0.778082,0.000000,1.000000,-0.984375,0.175537
4,0.0,0.477783,-0.065979,0.759766,-0.633301,0.771484,-0.687500,0.375,0.954452,0.707031,-0.707031,-0.282227,0.959473


In [11]:
GEOGRAPHICAL = True

frland = load_variable_at_time_and_level("MERRA2_101.const_2d_asm_Nx.00000000.nc4", variable="FRLAND",
                                         time=0, level=0, folder="raw").astype("float16")
frocean = load_variable_at_time_and_level("MERRA2_101.const_2d_asm_Nx.00000000.nc4", variable="FROCEAN",
                                          time=0, level=0, folder="raw").astype("float16")
frlake = load_variable_at_time_and_level("MERRA2_101.const_2d_asm_Nx.00000000.nc4", variable="FRLAKE",
                                         time=0, level=0, folder="raw").astype("float16")
frlandice = load_variable_at_time_and_level("MERRA2_101.const_2d_asm_Nx.00000000.nc4", variable="FRLANDICE",
                                            time=0, level=0, folder="raw").astype("float16")
phis = load_variable_at_time_and_level("MERRA2_101.const_2d_asm_Nx.00000000.nc4", variable="PHIS",
                                       time=0, level=0, folder="raw")
sgh = load_variable_at_time_and_level("MERRA2_101.const_2d_asm_Nx.00000000.nc4", variable="SGH",
                                      time=0, level=0, folder="raw")

phis -= phis.mean()
phis /= phis.std()
phis = phis.astype("float16")

sgh -= sgh.mean()
sgh /= sgh.std()
sgh = sgh.astype("float16")

indices = data[["lat", "lon"]].values.T

df["frland"] = frland[*indices]
df["frocean"] = frocean[*indices]
df["frlake"] = frlake[*indices]
df["frlandice"] = frlandice[*indices]
df["phis"] = phis[*indices]
df["sgh"] = sgh[*indices]

df.head()


(2, 10000000)


,lev,lat,lon,U_est,V_est,U,V,day %,year %,sin_day,cos_day,sin_year,cos_year,frland,frocean,frlake,frlandice,phis,sgh
0,0.0,-0.772461,-0.225708,2.115234,1.187500,2.119141,1.184570,0.500,0.839726,0.000000,-1.000000,-0.845215,0.534180,0.000000,1.000000,0.000000,0.0,-0.447754,-0.393799
1,0.0,-0.033325,0.590332,-0.853516,-0.666504,-0.840820,-0.603516,0.750,0.262329,-1.000000,-0.000000,0.997070,-0.077393,0.522949,0.477295,0.000000,0.0,-0.435303,-0.129272
2,0.0,0.649902,0.625000,0.499512,-0.630859,0.512207,-0.623535,0.625,0.297603,-0.707031,-0.707031,0.955566,-0.294678,0.996582,0.000000,0.003208,0.0,0.310303,1.616211
3,0.0,-0.855469,-0.083313,0.217407,1.049805,0.217773,1.053711,0.000,0.778082,0.000000,1.000000,-0.984375,0.175537,0.000000,0.000000,0.000000,1.0,1.767578,-0.242432
4,0.0,0.477783,-0.065979,0.759766,-0.633301,0.771484,-0.687500,0.375,0.954452,0.707031,-0.707031,-0.282227,0.959473,0.000000,1.000000,0.000000,0.0,-0.447754,-0.393799


In [15]:
df.dtypes

lev          float16
lat          float16
lon          float16
U_est        float16
V_est        float16
U            float16
V            float16
day %        float16
year %       float16
sin_day      float16
cos_day      float16
sin_year     float16
cos_year     float16
frland       float16
frocean      float16
frlake       float16
frlandice    float16
phis         float16
sgh          float16
dtype: object

In [16]:
# N - normalized
# G - geographical features
# CT - cyclic time

dataset = "N" if NORMALIZED else ""
dataset += "G" if GEOGRAPHICAL else ""
dataset += "CT" if CYCLIC_TIME else ""

df.to_feather(f"UV-{dataset}-{estimate_quantile}-{n}.ft")

In [17]:
df.astype("float32").describe()


,lev,lat,lon,U_est,V_est,U,V,day %,year %,sin_day,cos_day,sin_year,cos_year,frland,frocean,frlake,frlandice,phis,sgh
count,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07
mean,4.861209e-01,8.733363e-05,-1.608628e-03,1.357466e-03,3.005289e-04,-1.619807e-04,2.906788e-04,4.375165e-01,4.997328e-01,-6.621992e-05,-2.845820e-05,1.064984e-04,1.476460e-04,2.208213e-01,6.579908e-01,4.570988e-03,1.166172e-01,-1.563366e-04,2.166144e-04
std,2.885287e-01,5.788891e-01,5.772211e-01,9.992734e-01,9.932732e-01,1.000360e+00,9.996375e-01,2.864048e-01,2.886822e-01,7.070750e-01,7.070631e-01,7.072459e-01,7.069758e-01,3.983926e-01,4.615592e-01,2.942220e-02,3.149030e-01,9.995689e-01,1.000632e+00
min,0.000000e+00,-1.000000e+00,-1.000000e+00,-5.261719e+00,-8.234375e+00,-5.250000e+00,-8.234375e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-5.087891e-01,-3.937988e-01
25%,2.500000e-01,-5.000000e-01,-5.000000e-01,-6.621094e-01,-5.620117e-01,-6.635742e-01,-5.683594e-01,1.250000e-01,2.496338e-01,-7.070312e-01,-7.070312e-01,-7.070312e-01,-7.070312e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-4.477539e-01,-3.937988e-01
50%,5.000000e-01,0.000000e+00,0.000000e+00,-2.136230e-01,-1.167297e-02,-2.144775e-01,-1.149750e-02,5.000000e-01,4.997559e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,-4.477539e-01,-3.937988e-01
75%,7.221680e-01,5.000000e-01,5.000000e-01,4.868164e-01,5.468750e-01,4.868164e-01,5.532227e-01,7.500000e-01,7.500000e-01,7.070312e-01,7.070312e-01,7.070312e-01,7.070312e-01,9.796143e-02,1.000000e+00,0.000000e+00,0.000000e+00,-1.680908e-01,-1.483154e-01
max,9.721680e-01,1.000000e+00,9.965820e-01,7.855469e+00,8.562500e+00,7.886719e+00,8.437500e+00,8.750000e-01,9.995117e-01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,6.078125e+00,1.228906e+01
